### previously - workbook3 ;  in continuation - take preprocessSTG8_d1.csv
### Bert Prep
#### ---- set D1 - Business, Entertainment, Communication

In [1]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('preprocessSTG8_d1.csv')
df.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Calendar,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length
0,Coloring Book Barbaie,com.bisgumah.barbie,Entertainment,3.8,646456,False,this book can be used for all ages from childr...,https://docs.google.com/document/d/1kUDGpRA8fF...,0,0,...,0,0,0,1,0,0,0,1167,this book can be used for all ages from childr...,1136
1,Dj Car Sound And Sound Effects 2019,com.melonappsvpn.dj_carsounds,Entertainment,0.0,396,False,DJ Car Sound And Sound Effects 2019 is a car s...,https://www.hamootech.com/privacy_policy.php,0,0,...,0,0,0,0,0,0,0,340,dj car sound and sound effects is a car sound ...,330


In [3]:
df.shape

(189511, 22)

In [7]:
df.dropna(subset=['Clean_Description'],inplace=True)

In [4]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [5]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [6]:
!pip install transformers

In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [10]:
import numpy as np

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# Record the length of each sequence (after truncating to 512)
lengths = []

print('Tokenizing descriptions...')

# For every sentence...
for sent in df.Clean_Description:

    # Report progress
    if ((len(input_ids) % 20000) == 0):
        print('   Read {:,} descriptions.'.format(len(input_ids)))

    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

    # Record the truncated length.
    lengths.append(len(encoded_sent))

print('DONE. ')
print('{:>10,} comments'.format(len(input_ids)))

Token indices sequence length is longer than the specified maximum sequence length for this model (681 > 512). Running this sequence through the model will result in indexing errors


Tokenizing descriptions...
   Read 0 descriptions.
   Read 20,000 descriptions.
   Read 40,000 descriptions.
   Read 60,000 descriptions.
   Read 80,000 descriptions.
   Read 100,000 descriptions.
   Read 120,000 descriptions.
   Read 140,000 descriptions.
   Read 160,000 descriptions.
   Read 180,000 descriptions.
DONE. 
   189,447 comments


In [11]:
print(lengths)
print(len(lengths))
print(df.shape)

[226, 65, 40, 28, 41, 166, 88, 95, 77, 163, 18, 91, 28, 213, 174, 199, 380, 13, 62, 15, 332, 89, 106, 323, 7, 119, 34, 306, 74, 93, 14, 86, 347, 161, 106, 165, 7, 117, 23, 100, 170, 176, 197, 20, 213, 107, 203, 53, 28, 151, 313, 464, 120, 83, 71, 401, 97, 81, 31, 89, 31, 31, 441, 7, 166, 470, 65, 175, 59, 297, 6, 416, 11, 195, 131, 6, 45, 372, 324, 64, 22, 218, 39, 62, 167, 75, 68, 98, 80, 151, 335, 229, 177, 152, 135, 122, 52, 72, 13, 681, 126, 640, 271, 183, 24, 113, 338, 267, 269, 201, 8, 208, 156, 45, 123, 291, 176, 74, 445, 18, 156, 62, 154, 108, 211, 22, 104, 52, 194, 515, 48, 939, 108, 59, 327, 146, 88, 59, 20, 347, 133, 278, 126, 87, 74, 88, 379, 238, 212, 34, 25, 91, 41, 237, 453, 725, 31, 102, 386, 376, 112, 90, 140, 42, 110, 692, 393, 365, 25, 198, 404, 114, 123, 236, 10, 82, 66, 489, 54, 9, 60, 36, 228, 55, 27, 66, 109, 73, 48, 21, 346, 280, 156, 365, 39, 395, 450, 145, 145, 153, 34, 48, 183, 151, 8, 143, 277, 87, 206, 38, 34, 644, 166, 33, 40, 628, 280, 16, 132, 337, 159, 

In [12]:
df['token_length'] = lengths
df.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Coloring Book Barbaie,com.bisgumah.barbie,Entertainment,3.8,646456,False,this book can be used for all ages from childr...,https://docs.google.com/document/d/1kUDGpRA8fF...,0,0,...,0,0,1,0,0,0,1167,this book can be used for all ages from childr...,1136,226
1,Dj Car Sound And Sound Effects 2019,com.melonappsvpn.dj_carsounds,Entertainment,0.0,396,False,DJ Car Sound And Sound Effects 2019 is a car s...,https://www.hamootech.com/privacy_policy.php,0,0,...,0,0,0,0,0,0,340,dj car sound and sound effects is a car sound ...,330,65


In [13]:
print(sum(df['token_length'] < 25))

16625


In [14]:
print(sum(df['token_length'] >= 510))

9902


In [15]:
print('    Min length: {:,} tokens'.format(min(lengths)))
print('    Min length: {:,} tokens'.format(max(lengths)))
print(' Median length: {:,} tokens'.format(np.median(lengths)))

    Min length: 3 tokens
    Min length: 1,376 tokens
 Median length: 114.0 tokens


In [17]:
df.head(3)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Coloring Book Barbaie,com.bisgumah.barbie,Entertainment,3.8,646456,False,this book can be used for all ages from childr...,https://docs.google.com/document/d/1kUDGpRA8fF...,0,0,...,0,0,1,0,0,0,1167,this book can be used for all ages from childr...,1136,226
1,Dj Car Sound And Sound Effects 2019,com.melonappsvpn.dj_carsounds,Entertainment,0.0,396,False,DJ Car Sound And Sound Effects 2019 is a car s...,https://www.hamootech.com/privacy_policy.php,0,0,...,0,0,0,0,0,0,340,dj car sound and sound effects is a car sound ...,330,65
2,Creepy Story,com.creepypasta.club,Entertainment,0.0,22,False,Creepy Story is an app that contains scary sto...,http://hidayah.club/creepystory_privacy_policy...,0,0,...,0,0,1,0,0,0,171,creepy story is an app that contains scary sto...,171,40


In [ ]:
## save the df_d1 dataset file - 

In [19]:
df.to_csv("preprocessSTG9_d1.csv",index=False)